In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from time import time

%pylab inline
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

sys.path.append('..')

import isolation
import sample_players
import run_match
import my_baseline_player as custom
import book as b

from multiprocessing.pool import ThreadPool as Pool

Populating the interactive namespace from numpy and matplotlib


/Users/antelinvestigacionydesarrollo/anaconda3/envs/aind/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# Constant parameteres
time_limit = 150
depth = 4
full_search_depth = 2
matches_per_opening = 3

# Create the agents that will play
agent_names = ('CustomPlayer1', 'CustomPlayer2')
agent1 = isolation.Agent(custom.CustomPlayer, agent_names[0])
agent2 = isolation.Agent(custom.CustomPlayer, agent_names[1])
agents = (agent1, agent2)

# Get the initial states
root = isolation.isolation.Isolation()
opening_states = list(b.get_full_states(root, depth=full_search_depth))

# Generate the matches
matches = [(agents, state, time_limit, match_id) 
           for match_id, state in enumerate(opening_states)]
matches = matches * 3
print('Generated {} matches.'.format(len(matches)))

# Create or load the book
book = b.load_latest_book(depth=depth)

Generated 29403 matches.


In [3]:
num_processes = 4
batch_size = 20

# Small test for debugging
matches = matches[:100]

# Make some batches to save the results once in a while
x = matches
batches = [x[i*batch_size:(i+1)*batch_size] 
           for i in range(len(x) // batch_size + (len(x) % batch_size != 0))]

# Now fill the book!
pool = Pool(num_processes)

for batch in batches:
    tic = time.time()
    for result in pool.imap_unordered(isolation.play, batch):
        winner, game_history, match_id = result
        # print('Results for match {}: {} wins.'.format(match_id, winner.name))
        _, state, _, _ = matches[match_id]
        if state.locs[1] is not None:
            game_history.insert(0,state.locs[1])
        if state.locs[0] is not None:
            game_history.insert(0,state.locs[0])
        root = isolation.isolation.Isolation()
        b.process_game_history(root,
                               game_history, 
                               book,
                               agent_names.index(winner.name),
                               active_player=state.ply_count % 2,
                               depth=depth)
    toc = time.time()
    print('Batch time {} seconds.'.format((toc-tic)))
    b.save_book(book)
    b.remove_old_books()

Batch time 62.085216999053955 seconds.
Saving: book2018-06-13_15S35S46.714171.pkl
Batch time 57.09948515892029 seconds.
Saving: book2018-06-13_15S36S44.280247.pkl
rm book2018-06-13_15S35S46.714171.pkl
Batch time 60.6677770614624 seconds.
Saving: book2018-06-13_15S37S45.402935.pkl
rm book2018-06-13_15S36S44.280247.pkl
Batch time 61.89438199996948 seconds.
Saving: book2018-06-13_15S38S47.710156.pkl
rm book2018-06-13_15S37S45.402935.pkl
Batch time 53.73875713348389 seconds.
Saving: book2018-06-13_15S39S41.929056.pkl
rm book2018-06-13_15S38S47.710156.pkl


In [4]:
from isolation.isolation import Isolation

state = Isolation(board=20753973769799811568561646907385855, ply_count=2, locs=(15, 114))

In [5]:
state.player()

0

In [6]:
state.actions()

[<Action.NNE: 25>,
 <Action.ENE: 11>,
 <Action.WSW: -11>,
 <Action.WNW: 15>,
 <Action.NNW: 27>]

In [7]:
print(isolation.DebugState.from_state(state))


+ - + - + - + - + - + - + - + - + - + - + - +
| 2 |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   | 1 |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +



In [10]:
sum(abs(value) for value in book.values())

1075

In [9]:
29403*34 / (10*3600)  # Estimated time in hours

27.7695